# Final Project

- Saige Belanger
    - (20951877)
- Dylan Faelker
    - (20960747)
- Ethan Liu
    - (20959615)
- Timothy Zheng
    - t54zheng (20939203)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn import linear_model
import statsmodels.api as sm
import scipy.stats as stats
from math import sqrt

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import datetime as dt

import os.path

warnings.filterwarnings('ignore')

# Factors
We start with an initial list of factors from the provided list of 50 Factors in the ML examples.

TODO: Increase our breadth of factors to the category chosen by downloading and creating them, then merging them 

https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/

In [2]:
all_monthly_data = pd.read_sas("merged_df.sas7bdat", encoding = 'ISO-8859-1')

In [3]:
# Save all gvkeys - for WRDS Queries

# with open("gvkeys.txt", "w") as file:
#     for gvkey in set(all_monthly_data["gvkey"].dropna()):
#         file.write(f"{int(gvkey)},\n")

In [4]:
permnos = set(all_monthly_data["permno"])

In [5]:
gvkey_permno_dict = {}
for gvkey in set(all_monthly_data["gvkey"].dropna()):
    permno = all_monthly_data[all_monthly_data["gvkey"] == gvkey]["permno"].dropna().iloc[0]
    gvkey_permno_dict[gvkey] = permno

In [6]:
all_monthly_data.drop(["ticker", "conm", "gvkey", "cusip", "naics", "gsubind"], axis=1, inplace=True) # We don't use these columns anyway, drop them

In [7]:
factors = ['IM', 'range_20', 'log_vol_dollar_20',
       'range_120', 'log_vol_dollar_120', 'xret_5', 'xret_10', 'xret_20',
       'xret_indsize_20', 'xret_indsize_std20', 'xret_40', 'xret_120',
       'xret_indsize_120', 'xret_indsize_std120', 'KDJ_20', 'deviation_pct20',
       'MoneyFlowIndex_20', 'RSI_20', 'KDJ_120', 'deviation_pct120',
       'MoneyFlowIndex_120', 'RSI_120', 'IV_capm', 'mdr', 'ami_3', 'beta_3y',
       'beta_5y', 'tail_2y', 'dp', 'leverage', 'BL', 'roe', 'roa',
       'profitability', 'sales_g_q', 'sales_g_ttm', 'op_income_g_q', 'ni_g_q',
       'op_income_g_ttm', 'ni_g_ttm', 'sue_NI', 'BM', 'AM', 'EP', 'SP',
       'roe_q', 'roa_q', 'Cto', 'pe_ttm', 'lag_log_size']

ret_cols = ['ret_f1', 'ret_f2', 'ret_f3', 'ret_f4', 'ret_f5', 'ret_f6', 
            'ret_f7', 'ret_f8', 'ret_f9', 'ret_f10', 'ret_f11', 'ret_f12']

In [8]:
non_data_cols = [x for x in all_monthly_data.columns if x not in factors and x not in ret_cols]
non_data_cols

['permno', 'yyyymm', 'monthid', 'PRC', 'VOL', 'RET', 'SHROUT']

# Adding New Factors
* When you add a factor, document it here: [link](https://docs.google.com/spreadsheets/d/1rs9633QSYLVY5Z5DoGNy3USP2MROGtqTIKcbLG68wpE/edit#gid=1579135478) and fill properly
* Download the data file, if it's too large add it to the drive
* Also download the other files that arent on github but on the drive before working on this part of the notebook
    * https://drive.google.com/drive/u/0/folders/1D1eIYlkNxNLfzHJLzkGeE9ymr7doXg_6

## IMPORTANT NOTE - FACTOR/RETURN TIME
- When adding factors make sure you add such that factor is reported at t-1, **RET** has **T** returns (in same row)
- This means you need to download data from the range **(1979-12 to 2019-11)**

***

- Treasury and CPI Rates: [Link](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/index-treasury-and-inflation/us-treasury-and-inflation-indexes/)
- Federal Reserve Data: [Link](https://wrds-www.wharton.upenn.edu/pages/get-data/federal-reserve-bank-reports/interest-rates/data/)
- SEC Filings: https://wrds-www.wharton.upenn.edu/pages/get-data/wrds-sec-analytics-suite/wrds-sec-filings-queries/list-of-filings-exhibits/
- Analyzed Data: https://wrds-www.wharton.upenn.edu/pages/get-data/wrds-sec-analytics-suite/wrds-sec-text-analysis/readability-and-sentiment/

**TBD**
- Other Factors: Downloaded from https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/

In [9]:
# Add new generated factors here
factors += ["10M2", "volinc", "recession_affinity"]

# Recession Factor
- During recessions, companies that sell essential products/services typically outperform companies that offer products that are categorized as discretionary spending by consumers.
- Basically we want to come up with a factor such that during recessions, the factor is high for companies selling essential products/services and low for companies producing goods/services that are highly sensitive to recessions. Then we want the factor to be flipped when the company is out of a recession.
- We will categorize a recession as whenever the yield curve is inverted

### Proxy factors
- [10M2] Yield curve: US Treasury 10Y - US Treasury 2 Year: https://fred.stlouisfed.org/series/T10Y2YM
    - Recession if 10M2 < 0 [inverted yield curve]

- [volinc] Annual Income Volatility: Standard Deviation of annual net income growth [ni_g_ttm] for past 5 years, minimum past 1 year
    - Using trailing twelve month (TTM) measure because it avoids any seasonality considerations

In [10]:
all_monthly_data[all_monthly_data["permno"] == 86594.0][["ni_g_ttm", "yyyymm"]].dropna().head(12)

,ni_g_ttm,yyyymm
382792,-0.319421,200001.0
382793,-0.319421,200002.0
382794,-0.319421,200003.0
382795,-0.298619,200004.0
382796,-0.298619,200005.0
382797,-0.298619,200006.0
382798,0.319728,200007.0
382799,0.319728,200008.0
382800,0.319728,200009.0
382801,0.334539,200010.0


In [11]:
# Add annual income volatility - std of ni_g_ttm for past 5 years, minimum of past 1 year

volinc = {"yyyymm": [], "permno": [], "volinc": []}
for permno in permnos:
    ni_g_ttm = all_monthly_data[all_monthly_data["permno"] == permno][["ni_g_ttm", "yyyymm"]].dropna()
    date_range = sorted(list(ni_g_ttm["yyyymm"]))

    for i, yyyymm in enumerate(date_range):
        
        window = set(date_range[max(0, i-59):i+1]) # Look past 5 years (60 months)
        window_data = ni_g_ttm[ni_g_ttm["yyyymm"].isin(window)]
        
        if len(window_data) < 12:
            continue

        # Add std ni_g_ttm of past 5 years to volinc factor
        # Note this avoids lookahead bias because the data up to and including i
        # should be known (since ni_g_ttm is from i-1 as per data manual)
        volinc["yyyymm"].append(yyyymm)
        volinc["permno"].append(permno)
        volinc["volinc"].append(window_data["ni_g_ttm"].std())

volinc_df = pd.DataFrame(volinc)
volinc_df.head()

,yyyymm,permno,volinc
0,198012.0,49154.0,0.023537
1,198101.0,49154.0,0.024160
2,198102.0,49154.0,0.024472
3,198103.0,49154.0,0.024585
4,198104.0,49154.0,0.023754


In [12]:
all_monthly_data = pd.merge(all_monthly_data, volinc_df, on=["yyyymm", "permno"], how="outer")

## 10M2 Yield Curve

In [13]:
treasury_inflation = pd.read_sas("treasury_inflation.sas7bdat", encoding = 'ISO-8859-1')

In [14]:
fact_10M2 = treasury_inflation[["CALDT", "B2RET", "B10RET"]]

# Add one month to fit RET and factor time
fact_10M2["yyyymm"] = (fact_10M2["CALDT"] + pd.DateOffset(months=1)).dt.strftime("%Y%m").astype(float)

In [15]:
fact_10M2["10M2"] = fact_10M2["B10RET"] - fact_10M2["B2RET"]
fact_10M2

,CALDT,B2RET,B10RET,yyyymm,10M2
0,1979-12-31,0.005695,0.011951,198001.0,0.006256
1,1980-01-31,-0.000164,-0.037477,198002.0,-0.037313
2,1980-02-29,-0.036947,-0.050507,198003.0,-0.013560
3,1980-03-31,0.010329,0.048345,198004.0,0.038016
4,1980-04-30,0.084198,0.084375,198005.0,0.000177
...,...,...,...,...,...
477,2019-09-30,-0.001297,-0.013852,201910.0,-0.012555
478,2019-10-31,0.003274,-0.000742,201911.0,-0.004016
479,2019-11-29,-0.001038,-0.007410,201912.0,-0.006372
480,2019-12-31,0.002274,-0.011292,202001.0,-0.013566


In [16]:
all_monthly_data = pd.merge(all_monthly_data, fact_10M2[["yyyymm", "10M2"]], on="yyyymm")

## Recession Affinity
* Recession affinity is calculated as

- 1 / volinc **if 10M2 < 0** (recession)
- sqrt(volinc) $\times$ 550 **if 10M2 > 0** (no recession)
    - TBH Square root times 500 descision is arbitrary but it makes sense in "levelling" the playing field and lowering variance
    - Values when 10M2 < 0 and 10M2 > 0 are relatively equal

In [17]:
all_monthly_data["recession_affinity"] = np.where(all_monthly_data["10M2"] < 0, 1 / all_monthly_data["volinc"], 550 * np.sqrt(all_monthly_data["volinc"]))

In [18]:
test = all_monthly_data[["10M2", "volinc", "recession_affinity"]].dropna()

In [19]:
test[test["10M2"] < 0]["recession_affinity"].mean()

105.45824976257666

In [20]:
test[test["10M2"] > 0]["recession_affinity"].mean()

105.57830438214435

In [21]:
all_monthly_data

,permno,yyyymm,monthid,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,volinc,10M2,recession_affinity
0,10145.0,198001.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.070496,0.044878,-0.021226,0.065060,0.059729,-0.077586,-0.065421,NaN,0.006256,NaN
1,10241.0,198001.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.026766,-0.015625,-0.003968,0.007171,-0.044534,0.029661,0.003292,NaN,0.006256,NaN
2,10460.0,198001.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.087209,0.101604,0.014563,0.053140,0.252294,-0.073260,-0.115538,NaN,0.006256,NaN
3,10516.0,198001.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.033582,0.067181,0.036232,0.099324,0.081081,-0.071875,-0.158249,NaN,0.006256,NaN
4,10866.0,198001.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.070755,0.171806,0.038346,0.003676,0.007326,-0.082909,0.000000,NaN,0.006256,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440715,93374.0,201912.0,480.0,-0.147586,0.010868,17.310472,0.015246,17.388666,0.000246,-0.019599,...,-0.040206,0.062266,0.030582,-0.023207,-0.124141,0.086342,0.075351,0.008777,-0.006372,113.930650
440716,93419.0,201912.0,480.0,-0.147586,0.012941,16.775314,0.015804,16.961089,0.008011,0.004462,...,0.051303,-0.063196,-0.003818,-0.055366,-0.121751,0.349948,-0.066538,0.010539,-0.006372,94.884444
440717,93422.0,201912.0,480.0,-0.046054,0.059361,16.763782,0.067228,17.048398,0.015794,-0.033674,...,0.511246,0.139535,-0.115646,-0.305538,-0.003102,0.788889,0.484472,0.125235,-0.006372,7.984961
440718,93427.0,201912.0,480.0,0.260954,0.025465,16.473392,0.028009,16.544964,-0.005524,-0.006388,...,-0.023772,0.163569,-0.039240,-0.096733,-0.047755,0.138121,0.135851,0.016243,-0.006372,61.566110


***
# Data Cleanup
Done creating all factors, will clean up data before training step 

In [22]:
# Inputation - as in ML Lecture 1

# Drop NA in all non-numerical columns
all_monthly_data.dropna(subset=non_data_cols, inplace=True)

grouped_med = all_monthly_data.groupby(by='monthid')
# the lambda function gets the median per group in the groupby object, and fills the NaN values with the median per group
imputed_grouped = grouped_med.transform(lambda y: y.fillna(y.median()))

# This line assigns the values of the medians 
all_monthly_data = all_monthly_data.assign(**imputed_grouped.to_dict(orient='series'))
all_monthly_data.dropna(inplace=True)

In [23]:
# Filtering data by min price and min market share for each year

# Commenting out for runtime - does not drop any rows

# all_monthly_data['yyyy'] = all_monthly_data['yyyymm'].astype(str).str[:4]
# all_monthly_data['MKTSHR'] = all_monthly_data['PRC'] * all_monthly_data['SHROUT'] * 1_000

# to_drop_indices = []

# for permno in all_monthly_data.permno.unique():
#     for year in all_monthly_data['yyyy'].unique():
#         mask = (all_monthly_data['permno'] == permno) & (all_monthly_data['yyyy'] == year)
#         if all_monthly_data[mask].shape[0] != 0 != 0 and (all_monthly_data[mask]['MKTSHR'].iloc[0] < 100_000_000 or all_monthly_data[mask]['PRC'].iloc[0] <= 5):
#             to_drop_indices += list(all_monthly_data[mask].index)
# all_monthly_data.drop(to_drop_indices, inplace=True)

In [24]:
# Winsorizing factors--should winsorize the variables by quarter
for column in factors:
    for date in set(list(all_monthly_data["monthid"])):
        mask = (all_monthly_data["monthid"] == date)
        
        std = all_monthly_data[column][mask].std()
        mean = all_monthly_data[column][mask].mean()

        upper = mean + 3 * std
        lower = mean - 3 * std
        
        all_monthly_data[column][mask].clip(lower, upper, inplace= True)

In [25]:
all_monthly_data

,permno,yyyymm,monthid,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,volinc,10M2,recession_affinity
4588,10145.0,198012.0,12.0,0.374586,0.027456,15.617313,0.022920,15.145167,-0.045500,-0.051723,...,-0.006912,-0.002320,-0.093488,-0.135065,0.141141,0.002105,-0.066489,0.040110,0.013658,110.151075
4589,10241.0,198012.0,12.0,0.188068,0.018115,13.277532,0.019202,13.227823,-0.033571,-0.063374,...,-0.050445,-0.060000,-0.071186,-0.072993,0.152756,-0.034843,-0.007220,0.009316,0.013658,53.086666
4590,10460.0,198012.0,12.0,0.324759,0.036155,13.062007,0.027327,12.680238,0.077472,0.109122,...,-0.032727,-0.015152,-0.142308,-0.067265,0.106796,0.093860,0.085020,0.007682,0.013658,48.206830
4591,10516.0,198012.0,12.0,0.170149,0.031346,15.249913,0.031163,15.026468,-0.048646,-0.008587,...,-0.070968,0.055556,-0.072500,-0.126866,0.079316,0.087302,0.124088,0.014242,0.013658,65.636077
4592,10866.0,198012.0,12.0,0.273820,0.008009,11.612416,0.013282,11.810300,0.002714,-0.027249,...,0.046395,0.034954,-0.061674,0.066667,0.049107,0.004255,-0.023220,0.002783,0.013658,29.013979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440715,93374.0,201912.0,480.0,-0.147586,0.010868,17.310472,0.015246,17.388666,0.000246,-0.019599,...,-0.040206,0.062266,0.030582,-0.023207,-0.124141,0.086342,0.075351,0.008777,-0.006372,113.930650
440716,93419.0,201912.0,480.0,-0.147586,0.012941,16.775314,0.015804,16.961089,0.008011,0.004462,...,0.051303,-0.063196,-0.003818,-0.055366,-0.121751,0.349948,-0.066538,0.010539,-0.006372,94.884444
440717,93422.0,201912.0,480.0,-0.046054,0.059361,16.763782,0.067228,17.048398,0.015794,-0.033674,...,0.511246,0.139535,-0.115646,-0.305538,-0.003102,0.788889,0.484472,0.125235,-0.006372,7.984961
440718,93427.0,201912.0,480.0,0.260954,0.025465,16.473392,0.028009,16.544964,-0.005524,-0.006388,...,-0.023772,0.163569,-0.039240,-0.096733,-0.047755,0.138121,0.135851,0.016243,-0.006372,61.566110


## Factor Code

In [26]:
ff4_factors = pd.read_sas("ff4_factors.sas7bdat", encoding = 'ISO-8859-1')
ff4_factors["monthid"] = ff4_factors.index + 1
ff4_factors.head()

,DATEFF,SMB,HML,MKTRF,RF,UMD,monthid
0,1980-01-31,0.0162,0.0175,0.0551,0.0080,0.0755,1
1,1980-02-29,-0.0185,0.0061,-0.0122,0.0089,0.0788,2
2,1980-03-31,-0.0664,-0.0101,-0.1290,0.0121,-0.0955,3
3,1980-04-30,0.0105,0.0106,0.0397,0.0126,-0.0043,4
4,1980-05-30,0.0213,0.0038,0.0526,0.0081,-0.0112,5


In [27]:
dates = [int(x) for x in sorted(list(set(list(all_monthly_data["yyyymm"]))))]
dates[0], dates[-1]

(198012, 201912)

In [28]:
monthids = [int(x) for x in sorted(list(set(list(all_monthly_data["monthid"]))))]
monthids[0], monthids[-1], len(monthids)

(12, 480, 469)

In [29]:
testing_range = monthids[0:2*(len(monthids) // 3)]
validation_range = monthids[2 * len(monthids) // 3:]

# Validate that ranges have correct ratios
len(testing_range) / len(monthids), len(validation_range) / len(monthids), len(testing_range) + len(validation_range)

(0.6652452025586354, 0.3347547974413646, 469)

## Testing Factors

In [30]:
model_factors = ['IM', 'range_20', 'log_vol_dollar_20',
       'range_120', 'log_vol_dollar_120', 'xret_5', 'xret_10', 'xret_20',
       'xret_indsize_20', 'xret_indsize_std20', 'xret_40', 'xret_120',
       'xret_indsize_120', 'xret_indsize_std120', 'KDJ_20', 'deviation_pct20',
       'MoneyFlowIndex_20', 'RSI_20', 'KDJ_120', 'deviation_pct120',
       'MoneyFlowIndex_120', 'RSI_120', 'IV_capm', 'mdr', 'ami_3', 'beta_3y',
       'beta_5y', 'tail_2y', 'dp', 'leverage', 'BL', 'roe', 'roa',
       'profitability', 'sales_g_q', 'sales_g_ttm', 'op_income_g_q', 'ni_g_q',
       'op_income_g_ttm', 'ni_g_ttm', 'sue_NI', 'BM', 'AM', 'EP', 'SP',
       'roe_q', 'roa_q', 'Cto', 'pe_ttm', 'lag_log_size']

In [31]:
model_factors += ["10M2", "volinc", "recession_affinity"]

In [32]:
all_monthly_data = pd.merge(ff4_factors, all_monthly_data, on="monthid")

## [m, n, l] model for Fama-MacBeth Double Regression
We will use the technique employed during Assignment 2, utilizing a 36-month lookback for factor data to generate our betas (**First Stage**)
* For period $t_i$, we will use data starting at $t_{i-36} ... t_{i-1}$ if available. Worst case we look for 12 prior samples.

In [33]:
!pip install multiprocess

In [34]:
from multiprocess import Manager, cpu_count # You might have to change to multiprocessing if on windows
from multiprocess.pool import ThreadPool

In [35]:
# Threaded Approach
def add_betas(permno):
    results = []
    for (i, monthid) in enumerate(testing_range): 
        window = set(testing_range[max(0, i-35):i+1]) # t_(i-36) to t_(i-1) returns. Compare to t_i returns
        window_data = all_monthly_data[(all_monthly_data["permno"] == permno) & (all_monthly_data["monthid"].isin(window))]
        
        if len(window_data) < 12:
            continue

        explanatory_vars = window_data[model_factors + ["monthid"]]
        explanatory_vars.sort_values(by="monthid", inplace=True)
        explanatory_vars.set_index("monthid", inplace=True)
    
        explained_var = window_data[["monthid", "RET"]] # Since factors are from t-1
        explained_var.sort_values(by="monthid", inplace=True)
        explained_var.set_index("monthid", inplace=True)
        
        model = linear_model.LinearRegression().fit(explanatory_vars, 
                                                    explained_var["RET"])
        
        results.append({"monthid": monthid, 
                        "permno": permno, 
                        "RET": explained_var["RET"].iloc[-1], 
                    **{f"{factor}": model.coef_[i] for i, factor in enumerate(model_factors)}
                       })  
    return results

# UNCOMMENT THIS AND ADD TO LINE BELOW INSTEAD OF `permnos`
# FOR DEVELOPMENT - THIS CODE BLOCK TAKES LIKE 30 MINS TO RUN

# smaller_permno_list = list(permnos)[:10]

# Only compute if not in files (delete local copy of file if code above if modifying factors or code above)
if os.path.isfile("first_stage_df.csv"):
    first_stage_df = pd.read_csv("first_stage_df.csv", index_col=0)
else:
    # Runs once basically
    summary_results = []
    with ThreadPool(cpu_count() - 1) as P:
        summary_results = P.map(add_betas, permnos)
        summary_results = [item for sublist in summary_results for item in sublist]
        first_stage_df = pd.DataFrame(summary_results)
        
        # Save first stage df for easy loading
        first_stage_df.to_csv("first_stage_df.csv")
first_stage_df

,monthid,permno,RET,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,...,EP,SP,roe_q,roa_q,Cto,pe_ttm,lag_log_size,10M2,volinc,recession_affinity
0,23,49154.0,-0.044053,-0.638547,-0.030012,-0.230295,0.000042,0.179294,0.194247,0.245961,...,-0.000504,0.108639,0.027009,-0.004342,-0.003702,-0.063401,0.085833,-0.080160,0.000147,0.000788
1,24,49154.0,0.105991,-0.636234,-0.026982,-0.196861,0.000706,0.213877,0.205609,0.254094,...,-0.001213,0.140230,0.025653,-0.002634,-0.000659,-0.006820,0.074319,-0.081486,-0.000131,0.000196
2,25,49154.0,-0.037500,-0.551596,-0.055381,-0.039620,-0.002586,0.436495,0.063256,0.213124,...,0.000264,0.093481,0.025822,-0.008929,-0.032834,-0.104893,-0.192513,-0.162251,0.001401,-0.000459
3,26,49154.0,0.065801,-0.546921,-0.056979,-0.042687,-0.002952,0.453020,0.062154,0.208350,...,0.000379,0.090345,0.025240,-0.008983,-0.033347,-0.111269,-0.194868,-0.162358,0.001510,-0.000440
4,27,49154.0,0.102459,-0.434264,-0.070792,-0.038910,0.011427,0.504134,0.525319,0.373700,...,-0.000786,0.136573,0.040941,-0.011661,-0.044044,-0.059857,-0.209108,-0.247988,0.002238,0.000711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218148,319,81917.0,0.008880,-0.417166,27.133503,-0.687733,4.208687,0.575780,-0.502140,-13.827355,...,-1.385860,-3.568367,-1.397869,1.433513,-0.100285,-0.281886,0.459475,4.274448,-5.011145,-0.000944
218149,320,81917.0,0.063117,-1.498470,3.755503,-0.257776,3.675062,-0.445078,2.625505,2.184819,...,-0.197495,-6.360094,-8.197119,-3.366648,4.674314,-0.111389,-2.412481,1.036415,-1.653169,-0.001609
218150,321,81917.0,0.018998,-0.741329,9.007784,0.290788,0.647542,-0.219787,1.415487,13.258191,...,0.492021,-0.531365,-22.428783,-8.742253,-0.448722,0.071356,-1.334211,-2.494476,-0.971275,-0.001576
218151,322,81917.0,0.042373,-0.475296,9.021743,0.052722,-0.122259,0.005754,1.606962,18.922285,...,0.190179,-6.934000,-18.110909,-8.352385,-4.345289,0.106932,-3.053128,-4.745291,-0.399539,-0.000925


In [36]:
# Second stage regression
lambdas = {"monthid": []}
for factor in model_factors:
    lambdas[f"{factor}"] = []
    
for monthid in testing_range:
    monthid_returns = first_stage_df.loc[first_stage_df["monthid"] == monthid]

    # If empty
    if monthid_returns.empty:
        continue
    
    explanatory_vars = monthid_returns[model_factors + ["permno"]]
    explanatory_vars.sort_values(by="permno", inplace=True)
    explanatory_vars.set_index("permno", inplace=True)

    explained_var = monthid_returns[["permno", "RET"]]
    explained_var.sort_values(by="permno", inplace=True)
    explained_var.set_index("permno", inplace=True)
    
    model = linear_model.LinearRegression(n_jobs=len(model_factors)).fit(explanatory_vars, 
                                                                         explained_var["RET"])

    lambdas["monthid"].append(monthid)

    for (i, factor) in enumerate(model_factors):
        lambdas[factor].append(model.coef_[i])

In [37]:
second_stage_df = pd.DataFrame(lambdas)
second_stage_df

,monthid,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,xret_20,xret_indsize_20,...,EP,SP,roe_q,roa_q,Cto,pe_ttm,lag_log_size,10M2,volinc,recession_affinity
0,23,-0.075992,-0.862166,-0.117379,3.042904,0.038387,-0.009346,0.014427,-0.183057,0.143336,...,1.347061,-0.060104,-2.006185,5.889341,-0.401873,-0.047790,0.026378,0.397439,2.534419,2.236839
1,24,0.023343,-0.604369,0.000784,-0.285997,0.006769,-0.026912,-0.000170,0.005496,-0.031800,...,-0.050438,0.035577,-0.436685,0.900337,-0.054237,0.016148,-0.017756,-0.056440,-0.920704,-0.418451
2,25,-0.006402,-0.119707,-0.014524,2.083570,-0.024822,-0.045693,0.022848,0.000800,0.007226,...,-0.564819,-0.013840,-0.465363,1.303579,0.111374,-0.003357,-0.003174,-0.086227,1.247385,-0.304131
3,26,-0.008712,-0.212733,-0.021336,0.171175,0.032732,0.013792,-0.008538,0.033422,-0.044236,...,0.346069,-0.008789,0.158936,1.115967,-0.172607,0.011230,-0.006592,-0.005066,0.606262,-0.501671
4,27,-0.024970,-0.121119,-0.040795,0.705960,-0.019556,0.001229,0.016166,0.003649,0.046848,...,-0.367432,0.012085,0.181412,-0.200294,-0.100830,-0.003906,-0.028595,-0.005005,0.647293,-1.165565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,319,-0.007755,0.001205,0.024574,-0.000560,0.009582,0.001980,-0.000705,0.001858,0.002637,...,-0.000982,0.000857,-0.000370,-0.000222,0.000732,-0.002517,0.002756,-0.001874,-0.001340,1.406248
297,320,-0.002839,-0.000288,-0.016590,0.000461,0.002384,-0.001309,-0.000840,-0.000791,0.002315,...,-0.000944,-0.000019,-0.000511,0.004543,0.000580,0.009146,-0.002796,-0.000681,-0.001399,0.360405
298,321,0.000647,0.000135,-0.011179,0.000458,-0.009544,0.000907,-0.000451,-0.001221,-0.000597,...,0.000907,0.000116,0.000639,0.000505,-0.000155,0.014334,0.000663,0.001197,0.000849,1.354193
299,322,0.000026,-0.000588,-0.004611,-0.000500,-0.003722,-0.002754,-0.002179,-0.001635,-0.001676,...,-0.000383,-0.000855,0.000093,-0.000752,-0.000168,0.011501,-0.002741,-0.001645,0.001047,-1.068148


In [38]:
# Get p values
p_value_dict = {"factor": [], "p-value": []}
for factor in model_factors:
    lambdas = second_stage_df[factor]
    ttest = stats.ttest_ind(lambdas, np.zeros(len(lambdas))) # Compare to see if any lambdas are significantly different from zero
    p_value_dict['factor'].append(factor)
    p_value_dict['p-value'].append(ttest[1])

results_df = pd.DataFrame.from_dict(p_value_dict, orient='index')
results_df.round(2).T.sort_values(by="p-value")

,factor,p-value
49,lag_log_size,0.00737
42,AM,0.01486
18,KDJ_120,0.048546
1,range_20,0.049682
38,op_income_g_ttm,0.079787
51,volinc,0.122766
21,RSI_120,0.149815
3,range_120,0.18354
33,profitability,0.185649
41,BM,0.210322


# TODO
use these results to determine which factors to keep (among other considerations like cross-correlation, if they are in the same category, etc)

# Machine Learning

frick

# Performance Analysis

In [39]:
def total_ret(port_ret):
    return port_ret.sum()
    # return np.prod(port_ret + 1) - 1

def tracking_error(port_ret, bench_ret):
    return (port_ret - bench_ret).std()

def information_ratio(port_ret, bench_ret):
    return (total_ret(port_ret) - total_ret(bench_ret)) / tracking_error(port_ret, bench_ret)

def sharpe_ratio(port_ret, rf_ret):
    return information_ratio(port_ret, rf_ret)

def sharpe_ratio(port_xret):
    return total_ret(port_xret) / port_xret.std()

In [40]:
# Write Permnos - for WRDS Queries

# with open("permnos.txt", "w") as file:
#     for permno in permnos:
#         file.write(f"{int(permno)},\n")